# Tissue architectural features 

## Background and aim

In addition to intracellular changes, tumor progression is also accompanied by changes to the larger tissue structure. Here we aim to use the nucleus as a probe to characterize some of these changes. To this end, we focused on two key aspects: local crowding/cell density and long range tissue mechanics. Uninhibited proliferation of cancer cells as well as stromal recruitment are two factors that contribute to an increase in the density of cells in the local tumor microenvironment. Forces exerted by the expanding tumor mass on the stromal tissue and as well as increased ECM deposition and aligned collagen fibers in the tumor stroma are some examples of the changes in the local tissue mechanics during tumor progression.

Our samples are obtained from tissue microarray (TMA) slides containing annotated clinical biopsies from cancer patients as well as healthy individuals stained for DNA using Hoechst. The nuclei were segmented using a custom model (see _nuclear_feature_extraction_ notebook). In this notebook, we demonstrate how we characterise large scale changes to the tissue structure using the nucleus.

In [1]:
# import libraries
%load_ext autoreload
import sys
sys.path.append("../")
from pathlib import Path
from glob import glob
import pandas as pd
import os
from tqdm.notebook import tqdm

from src.utlis.Run_nuclear_local_density_estimation import local_nuclear_density
from src.utlis.Run_voronoi_features import extract_voronoi_features
from src.utlis.summarising_features import summarise_feature_table
from src.utlis.Run_mechanically_coupled_regions import mechanically_coupled_regions


In [2]:
## setup paths
path_to_rawimages='/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Raw_Images/all_cores/'
path_to_segmented_images="/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Segmented_labels/"

path_to_output_nuclear_local_density_features = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Local_density_features/"
path_to_output_voronoi_cell_features = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Voronoi_cell_features/"
path_to_output_mechanical_copuling_results = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Mechanically_coupled_regions/"
path_to_output_tissue_features = "/home/pathy_s/Documents/Mechanogenomic_Score_Breast_Cancer/Tissue_Summary/"
#create output directories if they do not exist
Path(path_to_output_nuclear_local_density_features).mkdir(parents=True, exist_ok=True)
Path(path_to_output_tissue_features).mkdir(parents=True, exist_ok=True)
Path(path_to_output_voronoi_cell_features).mkdir(parents=True, exist_ok=True)
Path(path_to_output_mechanical_copuling_results).mkdir(parents=True, exist_ok=True)

raw_image_dirs = sorted(glob(path_to_rawimages + "*.tif"))
seg_image_dirs = sorted(glob(path_to_segmented_images + "*.tif"))


## Local cell density / crowding

We measured local cell density via the following approaches 
(i) The number of neighbors for each nucleus at radii ~( 5,15, 25, 40 & 50)µm 
(ii) Distance between each nucleus and its kth nearest neighbor (k = 1,3,5,10,20) 
(iii) Voronoi cell properties for each nucleus (area, perimeter and eccentricity)

In [3]:
for i in tqdm(range(len(seg_image_dirs))):
    
    #measure voronoi cell features
    v_feature = extract_voronoi_features(seg_image_dirs[i])
    v_feature.to_csv(path_to_output_voronoi_cell_features + raw_image_dirs[i].rsplit('/', 1)[-1][:-4] + ".csv")
    
    #measure local density features
    d_feature = local_nuclear_density(seg_image_dirs[i])
    d_feature.to_csv(path_to_output_nuclear_local_density_features + raw_image_dirs[i].rsplit('/', 1)[-1][:-4] + ".csv")
    
    del d_feature,v_feature

  0%|          | 0/77 [00:00<?, ?it/s]

## Measure mechanically coupled regions 

We perform Density-based spatial clustering of applications with noise (DBSCAN) by using the angles, and centroid coordinates of the elongated nuclei to identify regions where all the nuclei are oriented in the same directions. We used a neighborhood radius ε of 400 pixels and minimum size of a cluster to be 15 nuclei. The identified clusters represent spatial neighbourhoods where nuclei are experiencing the same directional force i.e., they are mechanically coupled. We then calculate features that describe the clusters in tissues (Number of clusters, Fraction of nuclei clustered, area of clusters, relative distances between clusters etc). 


In [4]:
Rad=400
n_num=15
### Extract single nuclear features for all images and summarise at tissue level
for i in tqdm(range(len(seg_image_dirs))):
    try:
        temp = mechanically_coupled_regions(seg_image_dirs[i],
                                            path_to_output_mechanical_copuling_results
                                            ,Rad,n_num,True)
        del temp
    except:
        pass

  0%|          | 0/77 [00:00<?, ?it/s]